# Web Knowledge Source

Use **Web Knowledge Source** to retrieve public web information in real-time via Bing search without creating an index.

## 🔄 Workflow

```mermaid
flowchart LR
    subgraph Input["🌐 Input"]
        Query["User Query"]
    end
    
    subgraph Bing["🔍 Bing Grounding"]
        Search["Bing Search"]
        Filter["Domain Filtering"]
        Search --> Filter
    end
    
    subgraph Query["🤖 Query"]
        KB["Knowledge Base"]
        API["Agentic Retrieval API"]
        KB --> API
    end
    
    Query --> Search
    Filter --> KB
```

> 💡 Web Knowledge Source **does not create a local index**, directly retrieves real-time information via Bing search

## 📋 Table of Contents

| Step | Description | Jump to |
|------|-------------|---------|
| 0️⃣ Install Dependencies | Install necessary Python packages | [View](#install-deps) |
| 1️⃣ Initialize Configuration | Configure Azure AI Search and Azure OpenAI | [View](#init-config) |
| 2️⃣ Create Knowledge Source | Configure Bing search and domain filtering | [View](#step1) |
| 3️⃣ Create Knowledge Base | Create knowledge base | [View](#step2) |
| 4️⃣ Execute Query | Agentic Retrieval query | [View](#step3) |
| 5️⃣ View Activity Log | Check search process | [View](#step4) |
| 6️⃣ Multi-turn Conversation | Follow-up with context | [View](#step5) |
| 🧹 Cleanup Resources | Delete resources (optional) | [View](#step6) |

---

## 📊 Features

| Feature | Description |
|---------|-------------|
| 🌐 Real-time Search | Get latest web information via Bing |
| 🎯 Domain Filtering | Limit search scope to specified domains |
| ❌ No Index Required | Does not create local index, instantly available |
| 💬 Multi-turn Conversation | Supports context tracking |

## ⚠️ Important Notes

### Prerequisites
- Azure AI Search service in a [region supporting Agentic Retrieval](https://learn.microsoft.com/en-us/azure/search/search-region-support)
- [Semantic Ranker](https://learn.microsoft.com/en-us/azure/search/semantic-how-to-enable-disable) enabled
- Service in Azure public region (private cloud/sovereign cloud not supported)

### Cost and Data Notes
- Web Knowledge Source incurs additional fees, see [Bing Grounding Pricing](https://www.microsoft.com/en-us/bing/apis/grounding-pricing)
- Data is sent outside Azure compliance boundary (Bing service)

### Administrator Control
- If access is disabled, contact your Azure administrator
- Reference [Managing Web Knowledge Source Access](https://learn.microsoft.com/en-us/azure/search/agentic-knowledge-source-how-to-web-manage)

---

<a id="install-deps"></a>
## 0️⃣ Install Dependencies

In [ ]:
%pip install azure-search-documents==11.7.0b2 azure-identity python-dotenv -qU

<a id="init-config"></a>

## 1️⃣ Initialize Configuration

Configure Azure AI Search and Azure OpenAI connection information.

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import AzureCliCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient

load_dotenv()

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
gpt_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini")

index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

print("✅ Initialization successful")

python-dotenv could not parse statement starting at line 16


✅ 初始化成功


---

<a id="step1"></a>
## 2️⃣ Create Web Knowledge Source

Create a Web Knowledge Source and configure domain filtering rules.

### Configuration Description

| Parameter | Description |
|-----------|-------------|
| `allowed_domains` | List of allowed domains to search |
| `blocked_domains` | List of excluded domains |
| `include_subpages` | Whether to include subpages |

In [ ]:
# ============================================
# Step 1: Create Web Knowledge Source
# ============================================
from azure.search.documents.indexes.models import (
    WebKnowledgeSource,
    WebKnowledgeSourceParameters,
    WebKnowledgeSourceDomains
)

# Define names
web_ks_name = "demo2-web-knowledge-source"

# Create Web Knowledge Source
web_knowledge_source = WebKnowledgeSource(
    name=web_ks_name,
    description="Use Bing to search Microsoft and Azure official documentation",
    web_parameters=WebKnowledgeSourceParameters(
        domains=WebKnowledgeSourceDomains(
            # Allowed domains (Bing search will only search within these domains)
            allowed_domains=[
                {"address": "learn.microsoft.com", "include_subpages": True},
                {"address": "docs.microsoft.com", "include_subpages": True},
                {"address": "azure.microsoft.com", "include_subpages": True},
                {"address": "github.com/Azure", "include_subpages": True},
                {"address": "devblogs.microsoft.com", "include_subpages": True}
            ],
            # Blocked domains (optional, these domains will be excluded)
            blocked_domains=[
                {"address": "stackoverflow.com", "include_subpages": False}
            ]
        )
    )
)

# Create using API Key client
index_client.create_or_update_knowledge_source(knowledge_source=web_knowledge_source)
print(f"✅ Web Knowledge Source '{web_ks_name}' created successfully!")

✅ Web Knowledge Source 'demo2-web-knowledge-source' 创建成功！


---


<a id="step2"></a>
Knowledge Base is the entry point for queries, associated with Web Knowledge Source.

## 3️⃣ Create Knowledge Base

In [ ]:
from azure.search.documents.indexes.models import (
    KnowledgeBase,
    KnowledgeSourceReference,
    KnowledgeBaseAzureOpenAIModel,
    AzureOpenAIVectorizerParameters,
    KnowledgeRetrievalOutputMode,
    KnowledgeRetrievalLowReasoningEffort
)

web_kb_name = "demo2-web-knowledge-base"

aoai_kb_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    deployment_name=gpt_deployment,
    model_name=gpt_deployment
)

web_knowledge_base = KnowledgeBase(
    name=web_kb_name,
    description="Real-time knowledge base based on Bing search",
    knowledge_sources=[KnowledgeSourceReference(name=web_ks_name)],
    retrieval_instructions="Use Bing search to get the latest web information to answer questions",
    answer_instructions="Based on search results, provide accurate and timely answers with citations to information sources",
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_kb_params)],
    retrieval_reasoning_effort=KnowledgeRetrievalLowReasoningEffort()
)

index_client.create_or_update_knowledge_base(knowledge_base=web_knowledge_base)
print(f"✅ Knowledge Base '{web_kb_name}' created successfully!")

✅ Knowledge Base 'demo2-web-knowledge-base' 创建成功！


---


<a id="step3"></a>
Use Agentic Retrieval API to query the knowledge base and retrieve real-time information via Bing search.

## 4️⃣ Execute Query

In [ ]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import (
    KnowledgeBaseRetrievalRequest,
    KnowledgeBaseMessage,
    KnowledgeBaseMessageTextContent,
    WebKnowledgeSourceParams
)

web_kb_client = KnowledgeBaseRetrievalClient(
    endpoint=search_endpoint,
    knowledge_base_name=web_kb_name,
    credential=AzureKeyCredential(search_api_key)
)

question = "What are the latest features in Azure AI Search?"

request = KnowledgeBaseRetrievalRequest(
    include_activity=True,
    messages=[KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text=question)])],
    knowledge_source_params=[WebKnowledgeSourceParams(
        knowledge_source_name=web_ks_name,
        include_references=True,
        include_reference_source_data=True
    )]
)

result = web_kb_client.retrieve(retrieval_request=request)

print(f"🔍 Question: {question}\n")
print("📝 Answer:")
for resp in result.response:
    for content in resp.content:
        print(content.text)

🔍 问题: Azure AI Search 最新功能有哪些？

📝 答案:
Azure AI Searchの最新機能には、以下のような重要なアップデートがあります：

1. **ストレージとベクターインデックスの増加**: Azure AI Searchは、ストレージ容量とベクターインデックスサイズを大幅に増加させ、コストを増やすことなく、顧客が大規模な生成AIアプリケーションを運用できるようになりました。具体的には、ベクターインデックスサイズが最大11倍、ストレージが6倍、インデックス作成とクエリのスループットが2倍向上しています【ref_id:1】。

2. **ナレッジエージェントの名称変更**: ナレッジエージェントはナレッジベースに名称が変更され、REST APIのルートとプロパティに重要な変更が加わりました【ref_id:3】。

3. **新しいナレッジソースの種類**: インデックス付きOneLake、インデックス付きSharePoint、リモートSharePoint、Webなどの新しいナレッジソースが追加されました【ref_id:3】。

4. **効率的な取得操作**: 新しい推論作業や部分的な応答をサポートすることで、取得操作の効率が向上しました【ref_id:3】。

これらの機能は、Azure AI Searchを使用して生成AIアプリケーションを構築する際に、より高いスケーラビリティとパフォーマンスを提供します。


---


<a id="step4"></a>
Activity log shows the Bing search process, references contain URLs from search results.

## 5️⃣ View Activity Log and References

In [ ]:
import json

if result.activity:
    print("📊 Activity log:")
    for i, activity in enumerate(result.activity, 1):
        print(f"  [{i}] {json.dumps(activity.as_dict(), indent=4, ensure_ascii=False)}")

if result.references:
    print("\n🔗 Reference sources (Bing search results):")
    for i, ref in enumerate(result.references, 1):
        ref_dict = ref.as_dict()
        # Web references typically contain URL and title
        print(f"  [{i}] {ref_dict.get('title', 'N/A')}")
        print(f"      URL: {ref_dict.get('url', 'N/A')}")

📊 活动日志:
  [1] {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 1547,
    "input_tokens": 1472,
    "output_tokens": 58
}
  [2] {
    "id": 1,
    "type": "web",
    "elapsed_ms": 2206,
    "knowledge_source_name": "demo2-web-knowledge-source",
    "query_time": "2026-01-15T06:35:46.328Z",
    "count": 20,
    "web_arguments": {
        "search": "Azure AI Search 最新功能"
    }
}
  [3] {
    "id": 2,
    "type": "web",
    "elapsed_ms": 2374,
    "knowledge_source_name": "demo2-web-knowledge-source",
    "query_time": "2026-01-15T06:35:46.328Z",
    "count": 18,
    "web_arguments": {
        "search": "Azure AI Search 新特性"
    }
}
  [4] {
    "id": 3,
    "type": "web",
    "elapsed_ms": 420,
    "knowledge_source_name": "demo2-web-knowledge-source",
    "query_time": "2026-01-15T06:35:48.791Z",
    "count": 12,
    "web_arguments": {
        "search": "Azure AI Search 更新内容"
    }
}
  [5] {
    "id": 4,
    "type": "agenticReasoning",
    "reasoning_tokens": 43676,
    "

---

<a id="step5"></a>
## 6️⃣ Multi-turn Conversation Example

Web Knowledge Source supports multi-turn conversations, allowing follow-up questions with context tracking.

In [ ]:
def multi_turn_chat(kb_client, ks_name, messages):
    """
    Multi-turn conversation function
    
    Args:
        kb_client: KnowledgeBaseRetrievalClient instance
        ks_name: Knowledge Source name
        messages: Conversation message list, format [("user", "question"), ("assistant", "answer"), ...]
    """
    kb_messages = []
    for role, text in messages:
        kb_messages.append(KnowledgeBaseMessage(
            role=role,
            content=[KnowledgeBaseMessageTextContent(text=text)]
        ))
    
    request = KnowledgeBaseRetrievalRequest(
        include_activity=True,
        messages=kb_messages,
        knowledge_source_params=[WebKnowledgeSourceParams(
            knowledge_source_name=ks_name,
            include_references=True
        )]
    )
    
    return kb_client.retrieve(retrieval_request=request)

In [ ]:
# First turn conversation
conversation = [("user", "What is Azure AI Search?")]
result1 = multi_turn_chat(web_kb_client, web_ks_name, conversation)

answer1 = result1.response[0].content[0].text
print("🔹 Q: What is Azure AI Search?")
print(f"🔸 A: {answer1}\n")

# Second turn conversation (with context)
conversation.append(("assistant", answer1))
conversation.append(("user", "What languages does it support?"))
result2 = multi_turn_chat(web_kb_client, web_ks_name, conversation)

answer2 = result2.response[0].content[0].text
print("🔹 Q: What languages does it support?")
print(f"🔸 A: {answer2}")

---

<a id="step6"></a>
## 🧹 Cleanup Resources (Optional)

> 💡 Web Knowledge Source does not create a local index, deletion operation is very fast

In [ ]:
# Uncomment to delete resources
# index_client.delete_knowledge_base(web_kb_name)
# index_client.delete_knowledge_source(web_ks_name)
# print("✅ Resources deleted")

print("💡 To delete resources, uncomment the code above and run")

---

## 📊 Web Knowledge Source vs Blob Knowledge Source

| Aspect | Web KS | Blob KS |
|--------|--------|---------|
| **Data Source** | Public web | Private Blob |
| **Real-time** | ✅ Real-time search | ⏰ Requires indexing |
| **Index Creation** | ❌ Not created | ✅ Auto-created |
| **Data Control** | ❌ No control | ✅ Full control |
| **Use Cases** | Latest news | Private documents |

---

## 🔗 Related Links

- [03_knowledge_source_overview.ipynb](03_knowledge_source_overview.ipynb) - Overview and Navigation
- [03a_blob_knowledge_source.ipynb](03a_blob_knowledge_source.ipynb) - Blob Knowledge Source
- [03c_other_knowledge_sources.ipynb](03c_other_knowledge_sources.ipynb) - Other Knowledge Source Types